In [1]:
import os
import sys
import json
import openai
from langchain.tools import tool
from pydantic.v1 import BaseModel, Field
from IPython.display import display, HTML
from langchain.chat_models import ChatOpenAI
from langchain.schema.agent import AgentFinish
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad.log_to_messages import format_log_to_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
sys.path.append('../..')

In [2]:
# Setting your personal OPENAI Api Key
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
gpt_version = "gpt-4-0125-preview"

<h1> Initializing Components </h1>

**Critic LLM**

In [4]:
critic_model = ChatOpenAI(model_name=gpt_version, temperature=0)
critic_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an inspector who will receive a task as input. You only need to decide whether the task is fufilled or not so answer tersely"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
critic_chain = critic_prompt | critic_model | OpenAIFunctionsAgentOutputParser()

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


**Form Component**

*Templates*

In [27]:
# CSS and initialization LLM won't modify and doesn't need to see (To keep tokens in memory at a minimum)
form_css_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Contact Form</title>
<head>
<style>
@import url('https://fonts.googleapis.com/css?family=Open+Sans:400italic,400,300,600');

* {
	margin: 0;
	padding: 0;
	box-sizing: border-box;
	-webkit-box-sizing: border-box;
	-moz-box-sizing: border-box;
	-webkit-font-smoothing: antialiased;
	-moz-font-smoothing: antialiased;
	-o-font-smoothing: antialiased;
	font-smoothing: antialiased;
	text-rendering: optimizeLegibility;
}

body {
	font-family: "Open Sans", Helvetica, Arial, sans-serif;
	font-weight: 300;
	font-size: 12px;
	line-height: 30px;
	color: #777;
	background: #0CF;
}

.container {
	max-width: 400px;
	width: 100%;
	margin: 0 auto;
	position: relative;
}

#contact {
	background: #F9F9F9;
	padding: 25px;
	margin: 50px 0;
}

#contact h3, #contact h4 {
	margin: 5px 0 15px;
	display: block;
	font-size: 30px;
	font-weight: 400;
}

#contact h4 {
	font-size: 13px;
}

fieldset {
	border: medium none !important;
	margin: 0 0 10px;
	min-width: 100%;
	padding: 0;
	width: 100%;
}

#contact input, 
#contact textarea {
	width: 100%;
	border: 1px solid #CCC;
	background: #FFF;
	margin: 0 0 5px;
	padding: 10px;
}

#contact input:hover, 
#contact textarea:hover {
	-webkit-transition: border-color 0.3s ease-in-out;
	-moz-transition: border-color 0.3s ease-in-out;
	transition: border-color 0.3s ease-in-out;
	border: 1px solid #AAA;
}

#contact textarea {
	height: 100px;
	max-width: 100%;
	resize: none;
}

#contact button[type="submit"] {
	cursor: pointer;
	width: 100%;
	border: none;
	background: #0CF;
	color: #FFF;
	margin: 0 0 5px;
	padding: 10px;
	font-size: 15px;
}

#contact button[type="submit"]:hover {
	background: #09C;
	-webkit-transition: background 0.3s ease-in-out;
	-moz-transition: background 0.3s ease-in-out;
	transition: background-color 0.3s ease-in-out;
}

#contact button[type="submit"]:active {
	box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.5);
}

#contact input:focus, #contact textarea:focus {
	outline: 0;
	border: 1px solid #999;
}

::-webkit-input-placeholder {
	color: #888;
}

:-moz-placeholder {
	color: #888;
}

::-moz-placeholder {
	color: #888;
}

:-ms-input-placeholder {
	color: #888;
}
</style>
</head>
'''
# Html code the LLM will see
form_html_template = '''
<body>
<div class="container">  
  <form id="contact" action="" method="post">
    <h3> [Title] </h3>
    <h4> [Description] </h4>
    [Form-Fields]
    <fieldset>
      <button name="submit" type="submit" id="contact-submit" data-submit="...Sending">Submit</button>
    </fieldset>
  </form>
</div>
</body>
'''

*Prompts*

In [6]:
form_prompt_title = "Your task is to take the information in the provided messages and return an appropriate title for a form" +\
    " that will represent the information in scratchpad. Return only the title no other text"

In [7]:
form_prompt_description = "Your task is to take the information in the provided messages and return an appropriate one-sentence" +\
    " description of a form that represents all and only the information in scratchpad. Return only the description and no other text."

In [8]:
form_prompt_fields = '''Your task is to summarize and accurately represent the information in the provided messages as form fields.
    You are only allowed to use only one of two forms. 
    The first type is for short pieces of information and is of the format:
    <fieldset id="short-answer">
    	<text>(FILL-IN)[Title of the field]</text>
    	<input value="(FILL-IN)[Answer to field]" required>
    </fieldset>
    The other type is for longer pieces of information and is of the format:
    <fieldset id="text-answer">
    	<text>(FILL-IN)[Title of the text-field]</text>
    	<textarea value="(FILL-IN)[Answer to the text-field]" required></textarea>
    </fieldset>
    I want you to return an order list (no braces or seperators, just an ordered collection) of all the relevant pieces of information from the 
    scratchpad, and only those pieces of information as form fields. For each piece of information and accompanying type of form field you are
    only allowed to replace and fill-in the region which is of the format (FILL-IN)[...]. Do NOT modify any other part of the format but include
    it anyways (so include the <fieldset...></fieldset> parts too). Do NOT add any other html components such as <li> or <ol>.
    '''

In [9]:
critic_prompt = ''' Below you are provided a list. I want you to verify that the list has no braces or seperators (this means
    that a valid list of items i1 and i2 would be of the form 'i1i2') and is only composed of elements of the form: 
    <fieldset id="text-answer">
    	<text>...</text>
    	<textarea value="..." required></textarea>
    </fieldset>
    or:
    <fieldset id="short-answer">
    	<text>...</text>
    	<input value="..." required>
    </fieldset> .
    The inclusion of ... signifies we don't care what is in-between the parentheses. Answer Yes if the below is properly formatted and No if
    it is not. Do not answer with anything else and do not add any other information. You should strictly answer 'Yes' or 'No'. The list:
    '''

In [10]:
correction_prompt = '''
    The result you gave is improperly formatted. Look into your scratchpad for the proper formatting (Ignore any mentions of 'scratchpad'),
    I just want you to fix the formatting.
'''

*Chains*

In [11]:
form_model = ChatOpenAI(model_name=gpt_version, temperature=0)
form_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but terse assistant who follow's all the instructions given to them exactly."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
form_chain = form_prompt | form_model | OpenAIFunctionsAgentOutputParser()

Agent

In [32]:
def form_component():
    """
    Creates a UI-friendly form with all the information you desire.
    """
    pass
    

def critic_is_valid(result_string_output):
    check = critic_chain.invoke({
        "input": critic_prompt + result_string_output,
        "agent_scratchpad": format_log_to_messages([])
    })
    return check.log.lower() == 'yes'


def form_component_real(intermediate_steps, form_html_template):
    title = form_chain.invoke({"input": form_prompt_title, "agent_scratchpad": format_log_to_messages(intermediate_steps)}).log
    description = form_chain.invoke({"input": form_prompt_description, "agent_scratchpad": format_log_to_messages(intermediate_steps)}).log
    result = form_chain.invoke({
        "input": form_prompt_fields,
        "agent_scratchpad": format_log_to_messages(intermediate_steps)
    })
    intermediate_steps.append((result, form_prompt_fields))
    # Error detection via something similar to A2C but for LLMs
    while not critic_is_valid(result.log):
        print("Critic said the result is invalid. The result:")
        print(result.log)
        print(format_log_to_messages(intermediate_steps))
        result = form_chain.invoke({
                    "input": correction_prompt,
                    "agent_scratchpad": format_log_to_messages(intermediate_steps)
                })
    form_html_template = form_html_template.replace('[Title]', title)
    form_html_template = form_html_template.replace('[Description]', description)
    form_html_template = form_html_template.replace('[Form-Fields]', result.log)
    return form_css_template+form_html_template

<h1>Aggregating Components</h1>

In [23]:
components = [form_component]
functions = [convert_to_openai_function(f) for f in components]

<h1> Testing </h1>

**Initializing Test Chain**

In [24]:
test_model = ChatOpenAI(model_name=gpt_version, temperature=0)
test_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
test_chain = test_prompt | test_model | OpenAIFunctionsAgentOutputParser()

In [33]:
def test_form_agent():
    intermediate_steps=[]
    # Collect information for the form
    user_input = input("Prompt the LLM:")
    result = test_chain.invoke({
        "input": user_input,
        "agent_scratchpad": format_log_to_messages(intermediate_steps)
    })
    print(result.log)
    intermediate_steps.append((result,user_input))
    print(type(result))
    while not isinstance(result, AgentFinish):
        user_input = input("Prompt the LLM:")
        result = test_chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_log_to_messages((result,user_input))
        })
        print(result.log)
        intermediate_steps.append((result, user_input))
    # Run the cognitiveUI
    form_test_result = form_component_real(intermediate_steps, form_html_template)
    return form_test_result

In [34]:
form_test_result = test_form_agent()

Prompt the LLM: My name is Daniel


Hello, Daniel! How can I assist you today?
<class 'langchain_core.agents.AgentFinish'>


In [35]:
print(form_test_result)


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Contact Form</title>
<head>
<style>
@import url('https://fonts.googleapis.com/css?family=Open+Sans:400italic,400,300,600');

* {
	margin: 0;
	padding: 0;
	box-sizing: border-box;
	-webkit-box-sizing: border-box;
	-moz-box-sizing: border-box;
	-webkit-font-smoothing: antialiased;
	-moz-font-smoothing: antialiased;
	-o-font-smoothing: antialiased;
	font-smoothing: antialiased;
	text-rendering: optimizeLegibility;
}

body {
	font-family: "Open Sans", Helvetica, Arial, sans-serif;
	font-weight: 300;
	font-size: 12px;
	line-height: 30px;
	color: #777;
	background: #0CF;
}

.container {
	max-width: 400px;
	width: 100%;
	margin: 0 auto;
	position: relative;
}

#contact {
	background: #F9F9F9;
	padding: 25px;
	margin: 50px 0;
}

#contact h3, #contact h4 {
	margin: 5px 0 15px;
	display: block;
	font-size: 30px;
	font-weight: 400;
}

#contact h4 {
	font-si